# Database preprocessing for use by API

Import the libraries that will be used throughout the preprocessing.

In [92]:
import pandas as pd
import re
import sqlite3

Declare any constants that will be used in the preprocessing.

In [93]:
MOVIES_CSV_PATH = "../datasets/movies.csv";
RATINGS_CSV_PATH = "../datasets/ratings.csv";
DATABASE = "../../server/movielens.db"

 Read the datasets that will be processed.



In [94]:
df_movies = pd.read_csv(MOVIES_CSV_PATH)
df_ratings = pd.read_csv(RATINGS_CSV_PATH)

 Define a function that extracts the year from the movie titles using regular expressions.

In [95]:
def get_year_from_title(title: str):
    year = re.findall(r'\(\d{4}(?:–\d{4})?\)', title)


    if year:
        new_year = year[-1]
        new_year = str(new_year).replace("(", "").replace(")", "")
        return new_year

Create a function that removes the year from the movie titles.

In [96]:
def remove_year_from_title(title: str):
    new_title = re.sub(r'\(\d{4}(?:–\d{4})?\)', '', title)
    return new_title.strip()

Apply the function to extract all years from the movie titles and add them to a new column in the dataset.

Replace movies with no year information in the title with "Unknown."



In [97]:
df_movies["year"] = df_movies["title"].apply(get_year_from_title)
df_movies["year"].fillna("Unknown", inplace=True)
df_movies["year"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9742 entries, 0 to 9741
Series name: year
Non-Null Count  Dtype 
--------------  ----- 
9742 non-null   object
dtypes: object(1)
memory usage: 76.2+ KB


Remove the year from the movie titles in the dataset.


In [98]:
df_movies["title"] = df_movies["title"].apply(remove_year_from_title)
df_movies

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017
9739,193585,Flint,Drama,2017
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018


Calculate the average ratings along with the count of ratings and associate them with movie IDs.

In [99]:
mean_ratings = df_ratings.groupby("movieId")["rating"].agg(["count", "mean"]).reset_index()
mean_ratings.columns = ["movieId", "amount_ratings", "rating"]
mean_ratings


,movieId,amount_ratings,rating
0,1,215,3.920930
1,2,110,3.431818
2,3,52,3.259615
3,4,7,2.357143
4,5,49,3.071429
...,...,...,...
9719,193581,1,4.000000
9720,193583,1,3.500000
9721,193585,1,3.500000
9722,193587,1,3.500000


 Add the calculated columns to the movies dataset and replace any missing values with 0.

In [100]:
df_movies = df_movies.merge(mean_ratings, on="movieId", how="left")
df_movies.fillna(0, inplace=True)
df_movies

,movieId,title,genres,year,amount_ratings,rating
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,215.0,3.920930
1,2,Jumanji,Adventure|Children|Fantasy,1995,110.0,3.431818
2,3,Grumpier Old Men,Comedy|Romance,1995,52.0,3.259615
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,7.0,2.357143
4,5,Father of the Bride Part II,Comedy,1995,49.0,3.071429
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,2017,1.0,4.000000
9738,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,2017,1.0,3.500000
9739,193585,Flint,Drama,2017,1.0,3.500000
9740,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,2018,1.0,3.500000


Establish a connection to the database and execute insertion queries to insert the preprocessed data into the movies table.

In [101]:
connection = sqlite3.connect(DATABASE)

df_movies.to_sql("movies", con=connection, if_exists="replace", index=False)

connection.close()
